# CIFAR-10データセットの画像分類[分類] との比較

In [ ]:
# ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

# 適宜自分のApplicationPATHまで変更
APP_PATH = '/content/drive/Othercomputers/LAPTOP-3BHDM0TI/windows_research/workspace/research/carbon-steel-classification'

Mounted at /content/drive


In [ ]:
# 2.1 前準備

import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
from torch import optim

import cv2

In [ ]:
datasets_npz_x1 = np.load(APP_PATH + '/dataset_npz/x.1_dataset/sc_x10_960_x1.npz')
datasets_npz_x3 = np.load(APP_PATH + '/dataset_npz/x.1_dataset/sc_x10_960_x1.npz')
datasets_npz_all = np.load(APP_PATH + '/dataset_npz/x.1_dataset/sc_x10_960_x1.npz')

CATEGORIES=["S10C","S15C","S25C","S35C","S45C","S55C"]

print('x-type：', type(datasets_npz_x1['x']))
print('y-type：', type(datasets_npz_x1['y']))

print('x-次元数：', datasets_npz_x1['x'].ndim)
print('y-次元数：', datasets_npz_x1['y'].ndim)
print('x-size:', datasets_npz_x1['x'].size)
print('y-size:', datasets_npz_x1['y'].size)

x = datasets_npz_x1['x']
y = datasets_npz_x1['y']

print("before image size: {}".format(x[0].size()))    # 変換前画像サイズ

# xを標準化する
def stand(x):
    m=x.mean() #平均を計算
    s=x.std() #標準偏差を計算
    x=(x-m)/s #標準化の式
    return x

print('x標準化前')
print(x)
x=stand(x)
print('x標準化後')
print(x)

print("after image size: {}".format(x[0].size()))    # 変換前画像サイズ

print('==============================================')


# PytorchのTensor型へ変換
x = torch.tensor(x, dtype=torch.float32) # 元のデータセットは整数だから変える？
y = torch.tensor(y, dtype=torch.int64) # 目標値は分類と回帰でデータ型が異なる

print("tensor image size: {}".format(x[0].size()))    # 変換前画像サイズ

print(type(x), type(y))
print(x.shape, y.shape) # なんで横列が2つある？960, 960

dataset = torch.utils.data.TensorDataset(x, y)

num_train = int(len(dataset) * 0.8)
# num_val = int(len(dataset) * 0.2)
num_test = int(len(dataset) - num_train)

print('num_train:', num_train)
# print('num_val:', num_val)
print('num_test:', num_test)

# ランダムにするらめにシードの固定をした
torch.manual_seed(0)

# データセットの分割
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [num_train, num_test])

# print('train:', train_dataset)
# # print('val:', val)
# print('test:', test_dataset)

batch_size = 64


TypeError: ignored

In [ ]:
# 画像の確認

classes = ["S10C", "S15C", "S25C", "S35C", "S45C", "S55C"]

for i, image in enumerate(datasets_npz_x1['x']):
    if i == 9:
        break
    
    plt.subplot(3, 3, i+1)
    plt.imshow(image)
    plt.title(classes[datasets_npz_x1['y'][i]])
    plt.axis('off')
    print(i,'番目')

plt.show()



for i, image in enumerate(datasets_npz_x3['x']):
    if i == 9:
        break
    
    plt.subplot(3, 3, i+1)
    plt.imshow(image)
    plt.title(classes[datasets_npz_x3['y'][i]])
    plt.axis('off')
    print(i,'番目')

plt.show()



for i, image in enumerate(datasets_npz_all['x']):
    if i == 9:
        break
    
    plt.subplot(3, 3, i+1)
    plt.imshow(image)
    plt.title(classes[datasets_npz_all['y'][i]])
    plt.axis('off')
    print(i,'番目')

plt.show()

In [ ]:
# 2.2 訓練データとテストデータの用意

# # CIFAR10データセットの読み込み
# train_dataset = torchvision.datasets.CIFAR10(root='./data/',
#                                              train=True,
#                                              download=True,
#                                              transform=transforms.Compose([
#                                                  transforms.ToTensor(),
#                                                  transforms.Normalize(
#                                                      [0.5, 0.5, 0.5],
#                                                      [0.5, 0.5, 0.5],
#                                                  )
#                                              ]))
# test_dataset = torchvision.datasets.CIFAR10(root='./data/',
#                                             train=False,
#                                             download=True,
#                                             transform=transforms.Compose([
#                                                 transforms.ToTensor(),
#                                                 transforms.Normalize(
#                                                     [0.5, 0.5, 0.5],
#                                                     [0.5, 0.5, 0.5],
#                                                 )
#                                             ]))



# train_datasetの中身を確認
image, label = train_dataset[0]
print("image size: {}".format(image.size()))    # 画像サイズ　TODO：チャネルがなくなっている
print("label: {}".format(label))                # ラベルサイズ

In [ ]:
# ミニバッチサイズを指定したデータローダーを作成
train_batch = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=64,
                                          shuffle=True,
                                          num_workers=2)
test_batch = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=64,
                                         shuffle=False,
                                         num_workers=2)

# ミニバッチデータセットの確認
for images, labels in train_batch:
    print("batch images size: {}".format(images.size()))    # バッチの画像サイズ　TODO：チャネルがなくなっている
    print("image size: {}".format(images[0].size()))        # 1 枚の画像サイズ
    print("batch labels size: {}".format(labels.size()))    # バッチのラベルサイズ
    break

In [ ]:
# 画像の確認
# classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']  # CIFAR10のクラス

# for images, labels in train_batch:
#     for i in range(32):                 # 32枚の画像を表示
#         image = images[i] / 2 + 0.5     # 標準化を解除
#         image = image.numpy()           # Tensorからndarrayヘ
#         plt.subplot(4, 8, i+1)          # 4x8となるようにプロット
#         image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB) # 画像のスケールを変更
#         plt.imshow(np.transpose(image, (1, 2, 0)))  # matplotlibでは(縦, 横, チャンネル)の順
#         plt.title(CATEGORIES[labels[i]])   # ラベルをタイトルに
#         plt.axis('off') # 目盛を削除
#     plt.show()          # 表示
#     break

In [ ]:
# 2.3 ニューラルネットワークの定義

num_classes = 6    # CIFAR10のクラスの数を指定

class AlexNet(nn.Module):
    def __init__(self, num_classes=num_classes):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, padding=2),     # 畳み込み層
            nn.ReLU(inplace=True),                          # 活性化関数
            nn.MaxPool2d(kernel_size=2),                    # プーリング層
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),                   # ドロップアウト層
            nn.Linear(256 * 3* 3, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)                # 画像特徴量抽出パート
        x = x.view(x.size(0), 256 * 3 * 3)  # 3次元から1次元に変えて全結合層へ
        x = self.classifier(x)              # 画像分類パート
        return x
        
# ネットワークのロード
# CPUとGPUのどちらを使うかを指定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = AlexNet().to(device)
print(net)
# デバイスの確認
print("Device: {}".format(device))

In [ ]:
# 2.4 損失関数と最適化関数の定義

# 損失関数の定義
criterion = nn.CrossEntropyLoss()

# 活性化関数の定義
optimizer = optim.Adam(net.parameters())

In [ ]:
# 2.5 学習

# 損失と正解率を保持するリストを作成
train_loss_list = []        # 学習損失
train_accuracy_list = []    # 学習データの正答率
test_loss_list = []         # 評価損失
test_accuracy_list = []     # テストデータの正答率

# 学習(エポック)の実行
epoch = 10
for i in range(epoch):
    # エポックの進行状況を表示
    print("-------------------------------")
    print("Epoch: {}/{}".format(i+1, epoch))
    
    # 損失と正解率の初期化
    train_loss = 0      # 学習損失
    train_accuracy = 0  # 学習データの正答率
    test_loss = 0       # 評価損失
    test_accuracy = 0   # テストデータの正答率
    
    # ---------学習パート--------- #
    # ニューラルネットワークを学習モードに設定
    net.train()
    # ミニバッチごとにデータをロードし学習
    for images, labels in train_batch:
        # GPUにTensorを転送
        images = images.to(device)
        labels = labels.to(device)
        
        # 勾配を初期化
        optimizer.zero_grad()
        # データを入力して予測値を計算(順伝播)
        y_pred_prob = net(images)
        # 損失(誤差)を計算
        loss = criterion(y_pred_prob, labels)
        # 勾配の計算(逆伝搬)
        loss.backward()
        # パラメータ(重み)の更新
        optimizer.step()
        
        # ミニバッチごとの損失を蓄積
        train_loss += loss.item()
        
        # 予測したラベルを予測確率y_pred_probから計算
        y_pred_labels = torch.max(y_pred_prob, 1)[1]
        # ミニバッチごとに正解したラベル数をカウント
        train_accuracy += torch.sum(y_pred_labels == labels).item() / len(labels)
    
    # エポックごとの損失と正解率を計算(ミニバッチの平均の損失と正解率を計算)
    epoch_train_loss = train_loss / len(train_batch)
    epoch_train_accuracy = train_accuracy / len(train_batch)
    # ---------学習パートはここまで--------- #
    
    # ---------評価パート--------- #
    # ニューラルネットワークを評価モードに設定
    net.eval()
    # 評価時の計算で自動微分機能をオフにする
    with torch.no_grad():
        for images, labels in test_batch:
            # GPUにTensorを転送
            images = images.to(device)
            labels = labels.to(device)
            # データを入力して予測値を計算(順伝播)
            y_pred_prob = net(images)
            # 損失(誤差)を計算
            loss = criterion(y_pred_prob, labels)
            # ミニバッチごとの損失を蓄積
            test_loss += loss.item()
            
            # 予測したラベルを予測確率y_pred_probから計算
            y_pred_labels = torch.max(y_pred_prob, 1)[1]
            # ミニバッチごとに正解したラベル数をカウント
            test_accuracy += torch.sum(y_pred_labels == labels).item() / len(labels)
        
    # エポックごとの損失と正解率を計算(ミニバッチの平均の損失と正解率を計算)
    epoch_test_loss = test_loss / len(test_batch)
    epoch_test_accuracy = test_accuracy / len(test_batch)
    # ---------評価パートはここまで--------- #

    # エポックごとに損失と正解率を表示
    print("Train_Loss: {:.4f}, Train_Accuracy: {:.4f}".format(epoch_train_loss, epoch_train_accuracy))
    print("Test_Loss: {:.4f}, Test_Accuracy: {:.4f}".format(epoch_test_loss, epoch_test_accuracy))

    # 損失と正解率をリスト化して保存
    train_loss_list.append(epoch_train_loss)
    train_accuracy_list.append(epoch_train_accuracy)
    test_loss_list.append(epoch_test_loss)
    test_accuracy_list.append(epoch_test_accuracy)

In [ ]:
# 2.6 結果の可視化

# 損失
plt.figure()
plt.title('Train and Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.plot(range(1, epoch+1), train_loss_list, color='blue', linestyle='-', label='Train_Loss')
plt.plot(range(1, epoch+1), test_loss_list, color='red', linestyle='--', label='Test_Loss')
plt.legend()    # 凡例

# 正解率
plt.figure()
plt.title('Train and Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.plot(range(1, epoch+1), train_accuracy_list, color='blue', linestyle='-', label='Train_Accuracy')
plt.plot(range(1, epoch+1), test_accuracy_list, color='red', linestyle='--', label='Test_Accuracy')
plt.legend()    # 凡例

# 表示
plt.show()

In [ ]:
# 分類した画像を確認
# CIFAR10のクラス
# classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']  # CIFAR10のクラス

# ニューラルネットワークを評価モードに設定
net.eval()
# 評価時の計算で自動微分機能をオフにする
with torch.no_grad():
    for images, labels in test_batch:
        # GPUにTensorを転送
        images = images.to(device)
        labels = labels.to(device)
        # データを入力して予測値を計算(順伝播)
        y_pred_prob = net(images)
        # 予測したラベルを予測確率y_pred_probから計算
        y_pred_labels = torch.max(y_pred_prob, 1)[1]
        
        for i in range(9):
            image = images[i] / 2 + 0.5
            image = image.to('cpu').numpy()
            plt.subplot(3, 3, i+1)
            # matplotlibでは(縦, 横, チャネル)の順
            plt.imshow(np.transpose(image, (1, 2, 0)))
            plt.title(CATEGORIES[labels[i]])
            plt.title("{} ({})".format(CATEGORIES[y_pred_labels[i].item()], CATEGORIES[labels[i].item()]), color=('green' if y_pred_labels[i] == labels[i] else 'red'))
            plt.axis('off')
        plt.show()
        break
